![OurAirports logo](https://ourairports.com/images/logo-196px.png "OurAirports logo")
<div align="center">

## Scraping world airports data
</div>

Document explaining how world airports data is processed and saved in PostgreSQL database.

Link to data from Our Airports: [airports data data](https://davidmegginson.github.io/ourairports-data/airports.csv)

<hr>

# 1. Download and load data

## Data is in csv format.

In [ ]:
import pandas as pd

In [ ]:
airport_link = "https://davidmegginson.github.io/ourairports-data/airports.csv"

In [ ]:
airport_csv = pd.read_csv(airport_link)

In [ ]:
airport_csv.shape

In [ ]:
airport_csv

## Modify dataframe to display only columns that are valuable for our dataset.

In [ ]:
airport_csv.columns.values.tolist()

In [ ]:
filter_airports = airport_csv[
    [
        "id",
        "name",
        "type",
        "elevation_ft",
        "iso_country",
        "iso_region",
        "gps_code",
        "iata_code",
        "local_code",
        "home_link",
        "wikipedia_link",
        "latitude_deg",
        "longitude_deg",
    ]
]

## Change NaN values to None

In [ ]:
filter_airports = filter_airports.where(pd.notnull(filter_airports), None)

In [ ]:
filter_airports

# 3. Create foreign key for airport type if not in db

In [ ]:
from apps.civic_structure.models import Airport, AirportType

In [ ]:
airport_types = set(filter_airports["type"])

In [ ]:
airport_types

In [ ]:
[AirportType.objects.get_or_create(name=item) for item in airport_types if item]

# 4. Save in database

## Check state and country iso codes in database and airport for differences to prevent errors saving PK

In [ ]:
from apps.administrative_area.models import Country, State

In [ ]:
country_iso = set(Country.objects.order_by().values_list('iso2', flat=True))

In [ ]:
airport_country_iso = set(filter_airports["iso_country"].unique())

In [ ]:
state_iso = set(State.objects.order_by().values_list('iso_3166_1_2', flat=True))

In [ ]:
airport_state_iso = set(filter_airports["iso_region"].unique())

In [ ]:
country_diff = airport_country_iso.difference(country_iso)

In [ ]:
state_diff = airport_state_iso.difference(state_iso)

In [ ]:
country_diff

In [ ]:
state_diff

## Iterate through dataset then save or update rows in database

In [ ]:
from django.contrib.gis.geos import fromstr, MultiPoint

In [ ]:
for row in filter_airports.itertuples(index=False, name='Pandas'):
    
    if row.iso_country in country_diff:
        country = None
    else:
        country = Country.objects.filter(iso2=row.iso_country)[0]

    if row.iso_region in state_diff:
        state = None
    else:
        state = State.objects.filter(iso_3166_1_2=row.iso_region)[0]

    airport_type = AirportType.objects.filter(name=row.type)
    airport_type = airport_type[0] if airport_type else None
    
    point = MultiPoint(fromstr(f'POINT({row.longitude_deg} {row.latitude_deg})', srid=4326))

    updated_values = {
        "name": row.name,
        "airport_type": airport_type,
        "point": point,
        "state": state,
        "country": country,
        "iata_code": row.iata_code,
        "gps_code": row.gps_code,
        "local_code": row.local_code,
        "elevation_ft": row.elevation_ft,
        "url": row.home_link,
        "wikipedia_url": row.wikipedia_link,
    }
    
    # Update base on our_airports ID code
    Airport.objects.update_or_create(
        our_airports_id=row.id,
        defaults=updated_values,
    )